# Wildfire Hazard Potential for the United States (270-m), version 2023 (4th Edition)

Dataset source: https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_fileindex_RDS-2015-0047-4.html

Documentation: https://firelab.org/sites/default/files/2024-02/whp2023_cls_conus_metadata.pdf

https://data-usfs.hub.arcgis.com/datasets/usfs::wildfire-hazard-potential-version-2023-classified-image-service/explore?location=35.134247%2C-116.251988%2C6.95

https://www.fs.usda.gov/rds/archive/catalog/RDS-2015-0047-4


## Data dictionary

https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_metadata_RDS-2015-0047-4.html

In [17]:
pip install rioxarray

Note: you may need to restart the kernel to use updated packages.


In [18]:
import json
import os
import random

from glob import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rioxarray
import xarray as xr

from rasterio.enums import Resampling
from rasterio.errors import WindowError
from rasterio.features import geometry_mask
from rasterio.mask import mask
from rasterio.windows import Window

from shapely.geometry import Point, box

In [19]:
whp_filename = '../../wildfire-risk/data/Data-2/whp2023_GeoTIF/whp2023_cnt_conus.tif'

with rasterio.open(whp_filename) as src2:
    bounds = src2.bounds
    bbox = box(*bounds)
    geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=src2.crs)
    geo_json = [json.loads(geo.to_json())['features'][0]['geometry']]


RasterioIOError: ../../wildfire-risk/data/Data-2/whp2023_GeoTIF/whp2023_cnt_conus.tif: No such file or directory

In [ ]:
# Path to the folder containing .tif files
folder_path = "sim2real_layouts"

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))

layout_list = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[1].split('_E')[0]
    print(f"Processing {id}...")

    output_path = os.path.join("risk_layouts", f"whpi2023_cnt_{id}.tif")
    try:
        with rasterio.open(tif_file) as src1:
            # print the resolution of the raster in meters
            print(src1.res)
            out_image, out_transform = mask(src1, geo_json, crop=True)
            out_meta = src1.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Save cropped image
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
    except (ValueError, WindowError) as e:
        print(f"Error masking raster: {e}")
        
    print(f"Saved cropped intersection as {output_path}")

In [ ]:
tif_files = glob(os.path.join("risk_layouts", "*.tif"))
for tif_file in tif_files:
    # print(f"Processing {tif_file}...")
    with rasterio.open(tif_file) as src:
        wfpi_data = src.read(1)
    plt.imshow(wfpi_data, cmap='hot', vmin=0, vmax=500)
    plt.colorbar(label="WFPI Value")
    plt.title("Wind-enhanced Fire Potential Index (WFPI)")
    plt.show()

In [15]:
def convert_tif_to_npy(input_folder, output_folder):
    """
    Convert all .tif or .tiff files in a folder to .npy format.

    Args:
        input_folder (str): Path to the directory containing .tif/.tiff files.
        output_folder (str): Path to the directory where .npy files will be saved.
        max_files (int, optional): Maximum number of .tif files to process. If None, process all files.

    Yields:
        str: Path to each .npy file created.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file in os.listdir(input_folder):
        if file.lower().endswith('.tif') or file.lower().endswith('.tiff'):
            tif_path = os.path.join(input_folder, file)
            with rasterio.open(tif_path) as src:
                data = src.read(1)  # Read the first band
                # duplicate the data in 100 identical layers
                data = np.tile(data, (100, 1))
                print(data.shape)

            npy_filename = os.path.splitext(file)[0] + '.npy'
            np.save(os.path.join(output_folder, npy_filename), data)
            print(f"Converted {file} to {npy_filename}")

In [14]:
# move the .npy files from the risk_layouts folder to the appropriate folder
import shutil
import os
from glob import glob

# Path to the folder containing .tif files
folder_path = "../sim2real_layouts"

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))

layout_list = []
missing_layouts = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[2].split('_E')[0]
    print(id)
    rskfile_path = os.path.join("risk_layouts_npy", f"whpi2023_cnt_{id}.npy")
    if os.path.exists(rskfile_path):
        print("copying", rskfile_path, "to", os.path.join("../WideDataset", f"{id}/static_risk.npy"))
        shutil.copy(rskfile_path, os.path.join("../WideDataset", f"{id}/static_risk.npy"))
    else:
        missing_layouts.append(id)
print(len(missing_layouts), "layouts skipped, copied", len(tif_files) - len(missing_layouts), "layouts")


0319_04796
copying risk_layouts_npy/whpi2023_cnt_0319_04796.npy to ../WideDataset/0319_04796/static_risk.npy
0054_01387
copying risk_layouts_npy/whpi2023_cnt_0054_01387.npy to ../WideDataset/0054_01387/static_risk.npy
0277_02102
copying risk_layouts_npy/whpi2023_cnt_0277_02102.npy to ../WideDataset/0277_02102/static_risk.npy
0025_02019
copying risk_layouts_npy/whpi2023_cnt_0025_02019.npy to ../WideDataset/0025_02019/static_risk.npy
0110_02767
copying risk_layouts_npy/whpi2023_cnt_0110_02767.npy to ../WideDataset/0110_02767/static_risk.npy
0278_01323
copying risk_layouts_npy/whpi2023_cnt_0278_01323.npy to ../WideDataset/0278_01323/static_risk.npy
0049_01289
copying risk_layouts_npy/whpi2023_cnt_0049_01289.npy to ../WideDataset/0049_01289/static_risk.npy
0321_03136
copying risk_layouts_npy/whpi2023_cnt_0321_03136.npy to ../WideDataset/0321_03136/static_risk.npy
0065_03061
copying risk_layouts_npy/whpi2023_cnt_0065_03061.npy to ../WideDataset/0065_03061/static_risk.npy
0089_00984
copying 